In [1]:
#Importing required packages and files
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import os

#These imports were constructed
from wrangle import wrangle_zillow, train_validate

#Tools to build machine learning models and reports
from sklearn.model_selection import train_test_split

#Removes warnings and imporves asthenics
import warnings
#warnings.filterwarnings("ignore")

#Sets an option to show all columns
pd.options.display.max_columns = None
pd.options.display.width = 100
pd.options.display.max_colwidth = None


Source:
- https://www.kaggle.com/competitions/zillow-prize-1/overvie

Target column = taxvaluedollarcnt

Dropping all null values had a small affected on all fields except the upper outliers in 'taxvaluedollarcnt'/'tax_value'.
- Must adjust wrangle.py to see the results of the code below
- Measured by viewing the difference of a before and after DataFrame.describe()
- `after_df = before_df.dropna()`
- `change = round(before_df.describe - after_df.describe,2)`

In [2]:
df = wrangle_zillow()

In [3]:
schema = pd.read_excel('zillow_data_dictionary.xlsx')
#schema
schema.loc[[4, 3, 11, 51, 50, 54, 17]]

,Feature,Description
4,'bedroomcnt',Number of bedrooms in home
3,'bathroomcnt',Number of bathrooms in home including fractional bathrooms
11,'calculatedfinishedsquarefeet',Calculated total finished living area of the home
51,'taxvaluedollarcnt',The total tax assessed value of the parcel
50,'yearbuilt',The Year the principal residence was built
54,'taxamount',The total property tax assessed for that assessment year
17,'fips',Federal Information Processing Standard code - see https://en.wikipedia.org/wiki/FIPS_county_code for more details


In [4]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2131116 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   bedrooms    2131116 non-null  float64
 1   bathrooms   2131116 non-null  float64
 2   sqft        2131116 non-null  float64
 3   tax_value   2131116 non-null  float64
 4   year_built  2131116 non-null  float64
 5   tax_amount  2131116 non-null  float64
 6   fips        2131116 non-null  float64
dtypes: float64(7)
memory usage: 130.1 MB


# Plot the distributions
plt.figure(figsize=(16, 6))

# List of columns
cols = ['bedrooms', 'bathrooms', 'sqft', 'tax_value', 'year_built']

for col in cols:
    print(col)
    sns.boxplot(x=col, data=df)
    plt.show()

In [5]:
train, val, test = train_validate(df)

In [6]:
cols_outliers = ['bedrooms', 'bathrooms', 'sqft', 'tax_value', 'taxamount']

In [7]:
len(train)
#Output: 1198531


1193424

train = train[train['sqft'] <= train['sqft'].quantile(q=0.995)]

In [8]:
len(train)

1193424

for col in cols_outliers:
    df = df[df[col] <= df[col].quantile(q=0.995)]

In [9]:

train.quantile(q=.9999)

bedrooms      7.000000e+00
bathrooms     8.000000e+00
sqft          8.217658e+03
tax_value     4.899518e+06
year_built    2.015000e+03
tax_amount    5.471431e+04
fips          6.111000e+03
Name: 0.9999, dtype: float64

In [10]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
bedrooms,1193424.0,3.291420,0.914692,0.00,3.00,3.00,4.00,7.00
bathrooms,1193424.0,2.222555,0.943044,0.00,2.00,2.00,3.00,8.00
sqft,1193424.0,1838.509189,865.062613,1.00,1256.00,1619.00,2198.00,8407.00
tax_value,1193424.0,435420.536529,435471.495206,22.00,188450.75,327116.00,531000.00,6218405.00
year_built,1193424.0,1960.904146,22.099226,1801.00,1949.00,1958.00,1975.00,2016.00
tax_amount,1193424.0,5325.108997,4993.299569,13.36,2532.50,4098.86,6372.41,55708.43
fips,1193424.0,6048.322316,20.351067,6037.00,6037.00,6037.00,6059.00,6111.00
